## Kombinasi 7 :
- Delete duplicate
- Impute missing value using median/mode
- Outlier handling with winsorization
- Encoding
- Standard Scaler
- Feature selection with Decision Tree

In [71]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [72]:
df = pd.read_csv("../UFC_train.csv")

df.head()

,R_fighter,B_fighter,Referee,date,location,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,Winner
0,Joe Riggs,Joe Doerksen,Steve Mazzagatti,2004-08-21,"Las Vegas, Nevada, USA",False,Middleweight,NaN,NaN,NaN,...,0,0,0,Southpaw,182.88,177.80,185.0,26.0,21.0,Red
1,Jorge Masvidal,Al Iaquinta,Keith Peterson,2015-04-04,"Fairfax, Virginia, USA",False,Lightweight,1.156250,0.000000,0.394141,...,0,1,0,Orthodox,180.34,187.96,170.0,27.0,30.0,Blue
2,Dan Stittgen,Stephen Thompson,Josh Rosenthal,2012-02-04,"Las Vegas, Nevada, USA",False,Welterweight,NaN,NaN,NaN,...,0,0,0,Orthodox,185.42,NaN,170.0,28.0,31.0,Blue
3,Josh Koscheck,Johny Hendricks,Kevin Mulhall,2012-05-05,"East Rutherford, New Jersey, USA",False,Welterweight,0.695312,0.000000,0.783359,...,6,3,0,Orthodox,177.80,185.42,170.0,28.0,34.0,Blue
4,John Dodson,Manvel Gamburyan,James Warring,2016-04-16,"Tampa, Florida, USA",False,Bantamweight,0.500000,0.266602,0.381462,...,3,0,1,Orthodox,160.02,167.64,135.0,34.0,31.0,Red


In [73]:
# Delete duplicate
print(f"Shape before dropping duplicate : {df.shape}")
df.drop_duplicates(inplace=True)
print(f"Shape after dropping duplicate : {df.shape}")

Shape before dropping duplicate : (5410, 144)
Shape after dropping duplicate : (5410, 144)


In [74]:
def check_null(df):
    col_na = df.isnull().sum().sort_values(ascending=True)
    percent = col_na / len(df)
    missing_data = pd.concat([col_na, percent], axis=1, keys=['Total', 'Percent'])

    if (missing_data[missing_data['Total'] > 0].shape[0] == 0):
        print("Tidak ditemukan missing value pada dataset")
    else:
        print(missing_data[missing_data['Total'] > 0])

In [75]:
check_null(df)

                         Total   Percent
R_Weight_lbs                 2  0.000370
R_Height_cms                 4  0.000739
B_Weight_lbs                 8  0.001479
B_Height_cms                10  0.001848
R_Stance                    27  0.004991
...                        ...       ...
B_avg_GROUND_landed       1293  0.239002
B_avg_GROUND_att          1293  0.239002
B_avg_opp_CLINCH_landed   1293  0.239002
B_avg_TD_att              1293  0.239002
B_avg_HEAD_landed         1293  0.239002

[109 rows x 2 columns]


In [76]:
# Split string and number
# Split dataframe into object type and number type
df_object = df.select_dtypes(include='object')
df_number = df.select_dtypes(include=np.number)

In [77]:
# Impute missing value using median/mode
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')
df_imputed_number = imputer.fit_transform(df_number)
df_imputed_number = pd.DataFrame(df_imputed_number, columns=df_number.columns)

imputer = SimpleImputer(strategy='most_frequent')
df_imputed_object = imputer.fit_transform(df_object)
df_imputed_object = pd.DataFrame(df_imputed_object, columns=df_object.columns)

In [78]:
# Drop R_fighter, B_fighter, Referee, location, date
df_object = df_object.drop(['R_fighter', 'B_fighter', 'Referee', 'location', 'date'], axis=1)

In [79]:
# Combine df_object and df_imputed_number
df = pd.concat([df_object, df_imputed_number], axis=1)
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Middleweight,Orthodox,Southpaw,Red,0.015625,0.000000,0.450000,0.427500,0.250000,0.200000,...,0.0,0.0,0.0,0.0,0.0,182.88,177.80,185.0,26.0,21.0
1,Lightweight,Orthodox,Orthodox,Blue,1.156250,0.000000,0.394141,0.352422,0.239219,0.011484,...,0.0,4.0,0.0,1.0,0.0,180.34,187.96,170.0,27.0,30.0
2,Welterweight,Orthodox,Orthodox,Blue,0.015625,0.000000,0.450000,0.427500,0.250000,0.200000,...,0.0,0.0,0.0,0.0,0.0,185.42,182.88,170.0,28.0,31.0
3,Welterweight,Southpaw,Orthodox,Blue,0.695312,0.000000,0.783359,0.185547,0.088281,0.104375,...,1.0,5.0,6.0,3.0,0.0,177.80,185.42,170.0,28.0,34.0
4,Bantamweight,Orthodox,Orthodox,Red,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,...,0.0,2.0,3.0,0.0,1.0,160.02,167.64,135.0,34.0,31.0


In [80]:
check_null(df)

          Total   Percent
R_Stance     27  0.004991
B_Stance     64  0.011830


In [81]:
# Impute missing R_stance and B_stance with mode
df['R_Stance'].fillna(df['R_Stance'].mode()[0], inplace=True)
df['B_Stance'].fillna(df['B_Stance'].mode()[0], inplace=True)

In [82]:
check_null(df)

Tidak ditemukan missing value pada dataset


In [83]:
def check_outlier(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)

    # Menghitung RUB dan RLB.
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5*IQR
    upper_limit = Q3 + 1.5*IQR

    # Menampilkan banyaknya outlier pada atribut.
    outliers = (df < lower_limit) | (df > upper_limit)
    print ("Outlier pada tiap atribut:")
    print(outliers.sum())

    return outliers

In [84]:
# Split string and number
# Split dataframe into object type and number type
df_object = df.select_dtypes(include='object')
df_number = df.select_dtypes(include=np.number)

In [85]:
outliers = dict(check_outlier(df_number).sum())
print("\n\npercentage of outliers in each column:")
for key in outliers.keys():
    print(f"{key} = {outliers[key]/df_number[key].shape[0] * 100}%")

Outlier pada tiap atribut:
B_avg_KD                  561
B_avg_opp_KD             1142
B_avg_SIG_STR_pct         448
B_avg_opp_SIG_STR_pct     425
B_avg_TD_pct              186
                         ... 
R_Height_cms               19
R_Reach_cms                76
R_Weight_lbs              301
B_age                      31
R_age                      74
Length: 134, dtype: int64


percentage of outliers in each column:
B_avg_KD = 10.369685767097966%
B_avg_opp_KD = 21.109057301293902%
B_avg_SIG_STR_pct = 8.280961182994455%
B_avg_opp_SIG_STR_pct = 7.8558225508317925%
B_avg_TD_pct = 3.4380776340110906%
B_avg_opp_TD_pct = 6.210720887245841%
B_avg_SUB_ATT = 8.22550831792976%
B_avg_opp_SUB_ATT = 5.970425138632162%
B_avg_REV = 21.293900184842883%
B_avg_opp_REV = 23.696857670979668%
B_avg_SIG_STR_att = 6.358595194085027%
B_avg_SIG_STR_landed = 5.582255083179297%
B_avg_opp_SIG_STR_att = 6.22920517560074%
B_avg_opp_SIG_STR_landed = 6.598890942698706%
B_avg_TOTAL_STR_att = 4.787430683918669%
B_a

In [86]:
from scipy.stats.mstats import winsorize

# Identify columns with more than 0.5% outliers
outlier_columns = []
for key in outliers.keys():
    if outliers[key] / df_number[key].shape[0] > 0.005:
        outlier_columns.append(key)

# Winsorize the identified columns
for column in outlier_columns:
    df_number[column] = winsorize(df_number[column], limits=(0, 0.1))

# Combine df_object and df_number
df = pd.concat([df_object, df_number], axis=1)
df.head()


,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Middleweight,Orthodox,Southpaw,Red,0.015625,0.000000,0.450000,0.427500,0.250000,0.200000,...,0.0,0.0,0.0,0.0,0.0,182.88,177.80,185.0,26.0,21.0
1,Lightweight,Orthodox,Orthodox,Blue,0.649799,0.000000,0.394141,0.352422,0.239219,0.011484,...,0.0,4.0,0.0,1.0,0.0,180.34,187.96,170.0,27.0,30.0
2,Welterweight,Orthodox,Orthodox,Blue,0.015625,0.000000,0.450000,0.427500,0.250000,0.200000,...,0.0,0.0,0.0,0.0,0.0,185.42,182.88,170.0,28.0,31.0
3,Welterweight,Southpaw,Orthodox,Blue,0.649799,0.000000,0.586479,0.185547,0.088281,0.104375,...,1.0,4.0,4.0,2.0,0.0,177.80,185.42,170.0,28.0,34.0
4,Bantamweight,Orthodox,Orthodox,Red,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,...,0.0,2.0,3.0,0.0,0.0,160.02,167.64,135.0,34.0,31.0


In [87]:
outliers = dict(check_outlier(df_number).sum())
print("\n\npercentage of outliers in each column:")
for key in outliers.keys():
    print(f"{key} = {outliers[key]/df_number[key].shape[0] * 100}%")

Outlier pada tiap atribut:
B_avg_KD                  561
B_avg_opp_KD             1142
B_avg_SIG_STR_pct         202
B_avg_opp_SIG_STR_pct     158
B_avg_TD_pct                0
                         ... 
R_Height_cms               19
R_Reach_cms                35
R_Weight_lbs                0
B_age                       0
R_age                       4
Length: 134, dtype: int64


percentage of outliers in each column:
B_avg_KD = 10.369685767097966%
B_avg_opp_KD = 21.109057301293902%
B_avg_SIG_STR_pct = 3.7338262476894637%
B_avg_opp_SIG_STR_pct = 2.9205175600739373%
B_avg_TD_pct = 0.0%
B_avg_opp_TD_pct = 0.0%
B_avg_SUB_ATT = 0.0%
B_avg_opp_SUB_ATT = 0.0%
B_avg_REV = 21.293900184842883%
B_avg_opp_REV = 23.696857670979668%
B_avg_SIG_STR_att = 0.0%
B_avg_SIG_STR_landed = 0.0%
B_avg_opp_SIG_STR_att = 0.0%
B_avg_opp_SIG_STR_landed = 0.0%
B_avg_TOTAL_STR_att = 0.0%
B_avg_TOTAL_STR_landed = 0.0%
B_avg_opp_TOTAL_STR_att = 0.0%
B_avg_opp_TOTAL_STR_landed = 0.0%
B_avg_TD_att = 0.0%
B_avg_TD_lan

In [88]:
df['gender'] = df['weight_class'].apply(lambda x: 'women' if 'women' in x.lower() else 'male')

/var/folders/53/kv053qr13vxg2_y_1lg6hjg80000gn/T/ipykernel_52714/3282174900.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['gender'] = df['weight_class'].apply(lambda x: 'women' if 'women' in x.lower() else 'male')


In [89]:
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,gender
0,Middleweight,Orthodox,Southpaw,Red,0.015625,0.000000,0.450000,0.427500,0.250000,0.200000,...,0.0,0.0,0.0,0.0,182.88,177.80,185.0,26.0,21.0,male
1,Lightweight,Orthodox,Orthodox,Blue,0.649799,0.000000,0.394141,0.352422,0.239219,0.011484,...,4.0,0.0,1.0,0.0,180.34,187.96,170.0,27.0,30.0,male
2,Welterweight,Orthodox,Orthodox,Blue,0.015625,0.000000,0.450000,0.427500,0.250000,0.200000,...,0.0,0.0,0.0,0.0,185.42,182.88,170.0,28.0,31.0,male
3,Welterweight,Southpaw,Orthodox,Blue,0.649799,0.000000,0.586479,0.185547,0.088281,0.104375,...,4.0,4.0,2.0,0.0,177.80,185.42,170.0,28.0,34.0,male
4,Bantamweight,Orthodox,Orthodox,Red,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,...,2.0,3.0,0.0,0.0,160.02,167.64,135.0,34.0,31.0,male


In [90]:
df['weight_class'] = df['weight_class'].str.replace('Women', '')

In [91]:
# Get all weight_class values
weight_class = df['weight_class'].unique()
weight_class

array(['Middleweight', 'Lightweight', 'Welterweight', 'Bantamweight',
       'Flyweight', 'LightHeavyweight', 'Strawweight', 'Featherweight',
       'OpenWeight', 'Heavyweight', 'CatchWeight'], dtype=object)

In [92]:
# Create a dictionary to map weight_class values to numbers
weight_class_dict = {
    'CatchWeight' : 0,
    'Strawweight' : 1,
    'Flyweight' : 2,
    'Bantamweight' : 3,
    'Featherweight' : 4,
    'Lightweight' : 5,
    'Welterweight' : 6,
    'Middleweight' : 7,
    'LightHeavyweight' : 8,
    'Heavyweight' : 9,
    'OpenWeight' : 10,
}

gender_dict = {
    'male' : 1,
    'women' : 0
}

In [93]:
# Map each weight_class value to the correct number
df['weight_class'] = df['weight_class'].map(weight_class_dict)

In [94]:
# Map each gender value to the correct number
df['gender'] = df['gender'].map(gender_dict)

In [95]:
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,gender
0,7,Orthodox,Southpaw,Red,0.015625,0.000000,0.450000,0.427500,0.250000,0.200000,...,0.0,0.0,0.0,0.0,182.88,177.80,185.0,26.0,21.0,1
1,5,Orthodox,Orthodox,Blue,0.649799,0.000000,0.394141,0.352422,0.239219,0.011484,...,4.0,0.0,1.0,0.0,180.34,187.96,170.0,27.0,30.0,1
2,6,Orthodox,Orthodox,Blue,0.015625,0.000000,0.450000,0.427500,0.250000,0.200000,...,0.0,0.0,0.0,0.0,185.42,182.88,170.0,28.0,31.0,1
3,6,Southpaw,Orthodox,Blue,0.649799,0.000000,0.586479,0.185547,0.088281,0.104375,...,4.0,4.0,2.0,0.0,177.80,185.42,170.0,28.0,34.0,1
4,3,Orthodox,Orthodox,Red,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,...,2.0,3.0,0.0,0.0,160.02,167.64,135.0,34.0,31.0,1


In [96]:
# Get the remaining object columns
df_object = df.select_dtypes(include='object')

# Perform one-hot encoding
df_encoded = pd.get_dummies(df_object)

# Combine the encoded columns with the original dataframe
df = pd.concat([df, df_encoded], axis=1)

# Drop the original object columns
df.drop(df_object.columns, axis=1, inplace=True)

df.head()


,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,...,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Sideways,R_Stance_Southpaw,R_Stance_Switch,Winner_Blue,Winner_Draw,Winner_Red
0,7,0.015625,0.000000,0.450000,0.427500,0.250000,0.200000,0.15625,0.125000,0.000,...,0,0,0,0,0,1,0,0,0,1
1,5,0.649799,0.000000,0.394141,0.352422,0.239219,0.011484,0.15625,0.132812,0.000,...,0,0,0,1,0,0,0,1,0,0
2,6,0.015625,0.000000,0.450000,0.427500,0.250000,0.200000,0.15625,0.125000,0.000,...,0,0,0,1,0,0,0,1,0,0
3,6,0.649799,0.000000,0.586479,0.185547,0.088281,0.104375,0.09375,0.062500,0.000,...,1,0,0,1,0,0,0,1,0,0
4,3,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,1.06250,0.000244,0.125,...,0,0,0,1,0,0,0,0,0,1


In [97]:
# Export to CSV
# df.to_csv('UFC_kombinasi7_wt_featureselection.csv', index=False)

In [98]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

# Feature selection with Lasso
X = df.drop('B_Reach_cms', axis=1)
y = df['B_Reach_cms']

# Create the KNN model
lasso = Lasso(random_state=42)

lasso.fit(X, y)

# Get features with non-zero weights
non_zero_features = np.where(lasso.coef_ != 0)[0]

# Display the feature indices with non-zero weights
print("Features with non-zero weights:", non_zero_features)

Features with non-zero weights: [ 16  17  23  35  47  48  49  50  64  65  80  89 102 103 112 113 114 115
 130 131]


In [99]:
# Define the parameter grid
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0, 5.0],
    'fit_intercept': [True, False],
    'precompute': [True, False],
    'max_iter': [1000, 2000, 5000],
    'tol': [0.0001, 0.001, 0.01],
    'selection': ['cyclic', 'random']
}

# Create the GridSearchCV object
grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring='r2', n_jobs=-1)

# Fit the data to perform grid search
grid_search.fit(X, y)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best R-squared Score:", grid_search.best_score_)

/Users/azka/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 269.57382116305234, tolerance: 33.88967031008311
  model = cd_fast.enet_coordinate_descent_gram(
/Users/azka/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 148.7004544786323, tolerance: 33.54084028804991
  model = cd_fast.enet_coordinate_descent_gram(
/Users/azka/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.696e+02, tolerance: 3.389e+01
  model = cd_fast.enet_coordinate_descent(
/Users/azk

/Users/azka/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 53296.82510011723, tolerance: 14487.845217799973
  model = cd_fast.enet_coordinate_descent_gram(
/Users/azka/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 52795.51976806003, tolerance: 14488.120701119975
  model = cd_fast.enet_coordinate_descent_gram(
/Users/azka/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 51778.282821026965, tolerance: 14491.634887639973
  model = cd_fast.enet_coordinate_descent_gram(
/Users/azka/opt/anaconda3/lib/python3.9/site-packages/skl

Best Parameters: {'alpha': 0.1, 'fit_intercept': True, 'max_iter': 1000, 'precompute': True, 'selection': 'random', 'tol': 0.01}
Best R-squared Score: 0.7091990156529914


In [100]:
# Feature importance
feature_importance = grid_search.best_estimator_.coef_
print(feature_importance)

# Get the indices of the feature importance
feature_importance_indices = np.argsort(feature_importance)

[ 4.86525146e-01  0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
  1.52770441e-02  5.93838525e-03  4.47417535e-02 -6.60346895e-03
 -9.53206890e-03 -0.00000000e+00 -2.36087854e-02  2.60439973e-02
  0.00000000e+00 -5.36493094e-02 -0.00000000e+00  9.09517098e-03
  2.44157503e-03 -3.06939987e-02  0.00000000e+00  9.18256507e-03
 -0.00000000e+00 -0.00000000e+00 -9.97820729e-02 -4.48371480e-02
 -0.00000000e+00  5.19943003e-02  0.00000000e+00  8.53863917e-03
  0.00000000e+00 -3.97920474e-03 -2.44580954e-03 -2.46142585e-02
  0.00000000e+00 -3.32007000e-03  0.00000000e+00  0.00000000e+00
 -4.28109554e-02  4.31607331e-02  0.00000000e+00  2.37285334e-03
  1.68049118e-03  9.93299940e-04  2.56823643e-02 -0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  4.79108

In [101]:
print(len(feature_importance_indices))

148


In [102]:
non_zero_features = np.where(feature_importance != 0)[0]

non_zero_features_indices = np.argsort(feature_importance[non_zero_features])

print(len(non_zero_features_indices))

59


In [103]:
# df_feature_selection is df that has column with non-zero weight
df_feature_selection = df.iloc[:, non_zero_features]

df_feature_selection.head()

,weight_class,B_avg_SIG_STR_landed,B_avg_opp_SIG_STR_att,B_avg_opp_SIG_STR_landed,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_avg_opp_TOTAL_STR_landed,B_avg_TD_att,B_avg_opp_TD_att,B_avg_HEAD_att,...,R_avg_opp_GROUND_landed,R_avg_CTRL_time(seconds),R_avg_opp_CTRL_time(seconds),R_total_title_bouts,R_win_by_Decision_Unanimous,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,7,29.500000,64.000000,27.270996,89.625000,48.000000,44.500000,2.000000,2.038061,51.125000,...,2.687500,104.125000,89.531250,0.0,0.0,182.88,177.80,185.0,26.0,21.0
1,5,53.234375,89.953125,31.296875,141.210938,56.718750,33.078125,2.000000,1.835938,106.250000,...,1.093750,252.312500,115.000000,0.0,4.0,180.34,187.96,170.0,27.0,30.0
2,6,29.500000,64.000000,27.270996,89.625000,48.000000,44.500000,2.000000,2.038061,51.125000,...,2.687500,104.125000,89.531250,0.0,0.0,185.42,182.88,170.0,28.0,31.0
3,6,19.015625,37.226562,14.531250,51.500000,30.742188,28.671875,2.578125,1.625000,30.367188,...,0.226578,145.693754,116.292816,1.0,4.0,177.80,185.42,170.0,28.0,34.0
4,3,29.986572,93.299805,42.413086,102.170410,51.280762,71.359619,5.937500,3.682373,58.093994,...,0.093750,110.148438,162.796875,1.0,2.0,160.02,167.64,135.0,34.0,31.0


In [104]:
df_feature_selection['B_Reach_cms'] = y

/var/folders/53/kv053qr13vxg2_y_1lg6hjg80000gn/T/ipykernel_52714/3603037234.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feature_selection['B_Reach_cms'] = y


In [105]:
# Export to CSV
df_feature_selection.to_csv('UFC_kombinasi7_FS_lasso.csv', index=False)
